In [ ]:
# about
created by:
created on:

In [2]:
import pandas as pd
import numpy as np

# Pandas Ufuncs
* pandas does have an apply function and it is indeed faster than a for loop
* but ufuncs is even better!
    * it is most recommended using pandas builtin ufuncs for applying preprocessing tasks on columns (if a suitable ufunc is available for your task)
    
## What are ufuncs?
* special functions based on numpy library implemented in C -> they are highly efficient
* some exmaples
    * .diff
    * .shift
    * .cumsum
    * .cumcount
    * .str
    * .dt


## 1.  .str.split

* say you have a column of names (first+last) and you'd want to separate them into 2 columns
    * `df['name'] = df.name.str.split(" ",expand = True)`
    * use `expand=True` to get the new columns
    

## 2. unstack
* Unstack switches the rows to columns to get the activity counts as features. 
* By doing unstack we are transforming the last level of the index to the columns.

## 3. groupby, diff, shift, and loc + A great tip for efficiency
* Knowing the time differences between person activities can be quite interesting for predicting who is the most fun person. How long did a person hang out in a party? how long did he/she hang out at the the beach? This might be useful for us as a feature, depends on the activity.
* The most straight forward way to calculate the time differences would be to groupby the person name and them calculate the difference on the timestamp field using diff():

```
df = df.sort_values(by=['name','timestamp'])
df['time_diff'] = df.groupby('name')['timestamp'].diff()
```

* If you have a lot of data and you want to save some time (this can be about 10 times faster depends on your data size) you can skip the groupby and just do the diff after sorting the data and then deleting the first row of each person which is not relevant.

```
df = df.sort_values(by=['name','timestamp'])
df['time_diff'] = df['timestamp'].diff()
df.loc[df.name != df.name.shift(), 'time_diff'] = None

```


* to get the duration per row
`df[‘row_duration’] = df.time_diff.shift(-1)`

* .groupby.diff(periods, axis)
    * periods to shift for calculating difference. Int, default 1, accepts negative values.

* dataframe.shift(period=1, freq=none, axis=0, fill_value=<no default>)
    

  ## 3.1 diff()

In [15]:
df = pd.DataFrame({'a': [1, 2, 3, 4, 5, 6],
                   'b': [1, 1, 2, 3, 5, 8],
                   'c': [1, 4, 9, 16, 25, 36]})
df

,a,b,c
0,1,1,1
1,2,1,4
2,3,2,9
3,4,3,16
4,5,5,25
5,6,8,36


In [13]:
df.diff()

,a,b,c
0,NaN,NaN,NaN
1,1.0,0.0,3.0
2,1.0,1.0,5.0
3,1.0,1.0,7.0
4,1.0,2.0,9.0
5,1.0,3.0,11.0


In [14]:
df.diff(-1)

,a,b,c
0,-1.0,0.0,-3.0
1,-1.0,-1.0,-5.0
2,-1.0,-1.0,-7.0
3,-1.0,-2.0,-9.0
4,-1.0,-3.0,-11.0
5,NaN,NaN,NaN


In [16]:
df.diff(periods=1,axis=1)

,a,b,c
0,NaN,0,0
1,NaN,-1,3
2,NaN,-1,7
3,NaN,-1,13
4,NaN,0,20
5,NaN,2,28


  ## 3.2 shift()

In [9]:
df = pd.DataFrame({"Col1": [10, 20, 15, 30, 45],
                   "Col2": [13, 23, 18, 33, 48],
                   "Col3": [17, 27, 22, 37, 52]},
                  index=pd.date_range("2020-01-01", "2020-01-05"))
df

,Col1,Col2,Col3
2020-01-01,10,13,17
2020-01-02,20,23,27
2020-01-03,15,18,22
2020-01-04,30,33,37
2020-01-05,45,48,52


In [10]:
df.shift(periods=3)

,Col1,Col2,Col3
2020-01-01,NaN,NaN,NaN
2020-01-02,NaN,NaN,NaN
2020-01-03,NaN,NaN,NaN
2020-01-04,10.0,13.0,17.0
2020-01-05,20.0,23.0,27.0


In [6]:
df.shift(periods=1)

,Col1,Col2,Col3
2020-01-01,NaN,NaN,NaN
2020-01-02,10.0,13.0,17.0
2020-01-03,20.0,23.0,27.0
2020-01-04,15.0,18.0,22.0
2020-01-05,30.0,33.0,37.0


In [8]:
df.shift(periods=1,axis=1)

,Col1,Col2,Col3
2020-01-01,NaN,10,13
2020-01-02,NaN,20,23
2020-01-03,NaN,15,18
2020-01-04,NaN,30,33
2020-01-05,NaN,45,48


# 4. Cumcount and cumsum
* cumcount: creates a cumulative count
* umsum is just a cummulative summary of a numeric cell

# 5. Date ranges 
* date_range function returns dates incremented by days,months or years

In [17]:
# pd.date_range(start,end,frequency)
date_from = "2019-01-01"
date_to = "2019-01-12"
date_range = pd.date_range(date_from, date_to, freq="D")
date_range

DatetimeIndex(['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04',
               '2019-01-05', '2019-01-06', '2019-01-07', '2019-01-08',
               '2019-01-09', '2019-01-10', '2019-01-11', '2019-01-12'],
              dtype='datetime64[ns]', freq='D')

# 6. Nearest merge (merge_asof)

* used to merge dataframes by the nearest key (e.g. timestamp)
* example
    * the dataset quotes and trades:
    * The quotes DataFrame contains price changes for different stocks. Usually, there are many more quotes than trades.

In [18]:
quotes = pd.DataFrame(
    [
        ["2016-05-25 13:30:00.023", "GOOG", 720.50, 720.93],
        ["2016-05-25 13:30:00.023", "MSFT", 51.95, 51.96],
        ["2016-05-25 13:30:00.030", "MSFT", 51.97, 51.98],
        ["2016-05-25 13:30:00.041", "MSFT", 51.99, 52.00],
        ["2016-05-25 13:30:00.048", "GOOG", 720.50, 720.93],
        ["2016-05-25 13:30:00.049", "AAPL", 97.99, 98.01],
        ["2016-05-25 13:30:00.072", "GOOG", 720.50, 720.88],
        ["2016-05-25 13:30:00.075", "MSFT", 52.01, 52.03],
    ],
    columns=["timestamp", "ticker", "bid", "ask"],
)
quotes['timestamp'] = pd.to_datetime(quotes['timestamp'])

In [19]:
quotes

,timestamp,ticker,bid,ask
0,2016-05-25 13:30:00.023,GOOG,720.50,720.93
1,2016-05-25 13:30:00.023,MSFT,51.95,51.96
2,2016-05-25 13:30:00.030,MSFT,51.97,51.98
3,2016-05-25 13:30:00.041,MSFT,51.99,52.00
4,2016-05-25 13:30:00.048,GOOG,720.50,720.93
5,2016-05-25 13:30:00.049,AAPL,97.99,98.01
6,2016-05-25 13:30:00.072,GOOG,720.50,720.88
7,2016-05-25 13:30:00.075,MSFT,52.01,52.03


In [20]:
trades = pd.DataFrame(
    [
        ["2016-05-25 13:30:00.023", "MSFT", 51.95, 75],
        ["2016-05-25 13:30:00.038", "MSFT", 51.95, 155],
        ["2016-05-25 13:30:00.048", "GOOG", 720.77, 100],
        ["2016-05-25 13:30:00.048", "GOOG", 720.92, 100],
        ["2016-05-25 13:30:00.048", "AAPL", 98.00, 100],
    ],
    columns=["timestamp", "ticker", "price", "quantity"],
)
trades['timestamp'] = pd.to_datetime(trades['timestamp'])
trades

,timestamp,ticker,price,quantity
0,2016-05-25 13:30:00.023,MSFT,51.95,75
1,2016-05-25 13:30:00.038,MSFT,51.95,155
2,2016-05-25 13:30:00.048,GOOG,720.77,100
3,2016-05-25 13:30:00.048,GOOG,720.92,100
4,2016-05-25 13:30:00.048,AAPL,98.00,100


* let's merge trades and quotes by tickers -> largest quote can be 10ms behind the trade
*  If a quote is more than 10 ms behind the trade or there isn’t any quote, the bid and ask for that quote will be null (AAPL ticker in this example).

In [21]:
pd.merge_asof(trades, quotes, on="timestamp", by='ticker', tolerance=pd.Timedelta('10ms'), direction='backward')

,timestamp,ticker,price,quantity,bid,ask
0,2016-05-25 13:30:00.023,MSFT,51.95,75,51.95,51.96
1,2016-05-25 13:30:00.038,MSFT,51.95,155,51.97,51.98
2,2016-05-25 13:30:00.048,GOOG,720.77,100,720.50,720.93
3,2016-05-25 13:30:00.048,GOOG,720.92,100,720.50,720.93
4,2016-05-25 13:30:00.048,AAPL,98.00,100,NaN,NaN
